In [ ]:
import os
import pandas as pd
import numpy as np
import mindspore as ms
from mindspore import nn, Tensor
from mindspore.dataset import GeneratorDataset

# Set random seeds
ms.set_seed(42)
np.random.seed(42)

ms.set_context(mode=ms.PYNATIVE_MODE)

# -----------------------------
# LSTM Model
# -----------------------------
class LSTMModel(nn.Cell):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Dense(hidden_size, output_size)

        # Save architecture info for aggregation (same role as PyTorch attributes)
        self._input_size = input_size
        self._hidden_size = hidden_size
        self._num_layers = num_layers
        self._output_size = output_size

    def construct(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])


# -----------------------------
# Dataset (MindSpore Generator)
# -----------------------------
class PharmacyDataset:
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        x = np.array(self.sequences[idx], dtype=np.float32)  # (seq_len, features)
        y = np.array(self.targets[idx], dtype=np.float32)    # (features,)
        return x, y


# -----------------------------
# Load Data
# -----------------------------
def load_data(file_path, global_columns=None):
    data = pd.read_csv(file_path)

    data['addeddate'] = pd.to_datetime(
        data['addeddate'].astype(str).str.split().str[0],
        errors='coerce'
    )
    data.dropna(subset=['addeddate'], inplace=True)

    data['month'] = data['addeddate'].dt.to_period('M')
    data['medication_combination'] = data.groupby('Invoice')['name'].transform(
        lambda x: '+'.join(sorted(set(x)))
    )

    grouped_data = data.groupby(['month', 'medication_combination']).size().unstack(fill_value=0)

    if global_columns is not None:
        grouped_data = grouped_data.reindex(columns=global_columns, fill_value=0)

    return grouped_data


# -----------------------------
# Create Sequences
# -----------------------------
def create_sequences(data, seq_length=3):
    sequences, targets = [], []
    for i in range(len(data) - seq_length):
        sequences.append(data.iloc[i:i + seq_length].values)
        targets.append(data.iloc[i + seq_length].values)
    return sequences, targets


# -----------------------------
# Train Model
# -----------------------------
def train_model(model, dataset, loss_fn, optimizer, epochs=5):
    model.set_train(True)

    def forward_fn(x, y):
        preds = model(x)
        loss = loss_fn(preds, y)
        return loss

    grad_fn = ms.value_and_grad(forward_fn, None, optimizer.parameters)

    for epoch in range(epochs):
        epoch_loss = 0.0
        steps = 0

        for x_batch, y_batch in dataset.create_tuple_iterator():
            x_batch = Tensor(x_batch, ms.float32)  # (batch, seq_len, features)
            y_batch = Tensor(y_batch, ms.float32)  # (batch, features)

            loss, grads = grad_fn(x_batch, y_batch)
            optimizer(grads)

            epoch_loss += float(loss.asnumpy())
            steps += 1

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / max(steps, 1):.4f}")


# -----------------------------
# Aggregate Models (Average weights)
# -----------------------------
def aggregate_models(models):
    if not models:
        return None

    input_size = models[0]._input_size
    hidden_size = models[0]._hidden_size
    num_layers = models[0]._num_layers
    output_size = models[0]._output_size

    agg_model = LSTMModel(input_size, hidden_size, num_layers, output_size)

    # Collect params as numpy
    param_lists = []
    for m in models:
        params = {p.name: p.data.asnumpy() for p in m.get_parameters()}
        param_lists.append(params)

    # Average
    avg_params = {}
    for name in param_lists[0].keys():
        avg_params[name] = np.mean([pl[name] for pl in param_lists], axis=0)

    # Load averaged weights
    for p in agg_model.get_parameters():
        if p.name in avg_params:
            p.set_data(Tensor(avg_params[p.name], dtype=p.data.dtype))

    return agg_model


# -----------------------------
# Predict Trends
# -----------------------------
def predict_trends(model, data, seq_length=3, steps=3):
    model.set_train(False)

    inputs = Tensor(data.iloc[-seq_length:].values.astype(np.float32), ms.float32)
    inputs = inputs.expand_dims(0)  # (1, seq_len, features)

    predictions = []
    for _ in range(steps):
        pred = model(inputs).asnumpy()  # (1, features)
        predictions.append(pred)

        pred_tensor = Tensor(pred.astype(np.float32), ms.float32).expand_dims(1)  # (1, 1, features)
        inputs = ms.ops.concat((inputs[:, 1:, :], pred_tensor), axis=1)

    return np.array(predictions)


# -----------------------------
# Save Predictions (to Real_LSTM_Result)
# -----------------------------
def save_predictions(level_models, level_data, level, result_folder):
    if level_models[level] is None:
        return

    if isinstance(level_models[level], list):
        for i, model in enumerate(level_models[level]):
            if level not in level_data or i >= len(level_data[level]):
                continue

            predictions = predict_trends(model, level_data[level][i])
            all_medications = level_data[level][i].columns
            pred_df = pd.DataFrame(predictions.squeeze(), columns=all_medications)

            output_path = os.path.join(result_folder, f"{level}_predictions_{i}.csv")
            pred_df.to_csv(output_path, index=False)
            print(f"Saved {level} predictions → {output_path}")
    else:
        if level not in level_data or level_data[level] is None:
            return

        predictions = predict_trends(level_models[level], level_data[level])
        all_medications = level_data[level].columns
        pred_df = pd.DataFrame(predictions.squeeze(), columns=all_medications)

        output_path = os.path.join(result_folder, f"{level}_predictions.csv")
        pred_df.to_csv(output_path, index=False)
        print(f"Saved {level} predictions → {output_path}")


# -----------------------------
# Main Execution
# -----------------------------
if __name__ == "__main__":
    output_folder = "./outputDateUni/"   # INPUT real dataset folder (unchanged)
    cities = ["City1", "City2", "City3"]
    zones_per_city = 3
    pharmacies_per_zone = 4
    seq_length = 3

    # OUTPUT folder for results (NEW)
    result_folder = "./Real_LSTM_Result/"
    os.makedirs(result_folder, exist_ok=True)

    level_models = {'pharmacy': [], 'zone': [], 'city': [], 'national': None}
    level_data = {'zone': [], 'city': [], 'national': None}

    # Step 1: Collect all medication combinations to create a global column set
    all_columns = set()
    for city in cities:
        for zone in range(1, zones_per_city + 1):
            for pharmacy in range(1, pharmacies_per_zone + 1):
                pharmacy_path = os.path.join(
                    output_folder, city, f"Zone{zone}",
                    f"Ph{pharmacy:02d}_Z{zone:02d}_C{cities.index(city) + 1:02d}.csv"
                )
                if os.path.exists(pharmacy_path):
                    data = pd.read_csv(pharmacy_path)
                    data['medication_combination'] = data.groupby('Invoice')['name'].transform(
                        lambda x: '+'.join(sorted(set(x)))
                    )
                    all_columns.update(data['medication_combination'].unique())

    global_columns = sorted(all_columns)

    # Step 2: Train models with aligned columns (Zone-level training)
    for city in cities:
        for zone in range(1, zones_per_city + 1):
            zone_data = []

            for pharmacy in range(1, pharmacies_per_zone + 1):
                pharmacy_path = os.path.join(
                    output_folder, city, f"Zone{zone}",
                    f"Ph{pharmacy:02d}_Z{zone:02d}_C{cities.index(city) + 1:02d}.csv"
                )
                if os.path.exists(pharmacy_path):
                    df = load_data(pharmacy_path, global_columns=global_columns)
                    zone_data.append(df)

            if not zone_data:
                continue

            merged_data = sum(zone_data) / len(zone_data)
            sequences, targets = create_sequences(merged_data, seq_length)
            if not sequences:
                continue

            dataset_obj = PharmacyDataset(sequences, targets)
            dataset = GeneratorDataset(dataset_obj, column_names=["x", "y"], shuffle=True).batch(32)

            model = LSTMModel(
                input_size=merged_data.shape[1],
                hidden_size=64,
                num_layers=2,
                output_size=merged_data.shape[1]
            )

            optimizer = nn.Adam(model.trainable_params(), learning_rate=0.001)
            criterion = nn.MSELoss()

            print(f"Training Zone {zone} in {city}...")
            train_model(model, dataset, criterion, optimizer, epochs=2)

            level_models['zone'].append(model)
            level_data['zone'].append(merged_data)

    # Step 3: Aggregate models at the city level
    for i in range(0, len(level_models['zone']), 3):
        city_models = level_models['zone'][i:i + 3]
        city_data = level_data['zone'][i:i + 3]
        if city_models:
            level_models['city'].append(aggregate_models(city_models))
            level_data['city'].append(sum(city_data) / len(city_data))

    # Step 4: Aggregate models at the national level
    if level_models['city']:
        level_models['national'] = aggregate_models(level_models['city'])
        level_data['national'] = sum(level_data['city']) / len(level_data['city'])

    # Step 5: Save predictions (to Real_LSTM_Result)
    for level in level_models:
        save_predictions(level_models, level_data, level, result_folder)


Training Zone 1 in City1...
Epoch 1/2, Loss: 0.0213
Epoch 2/2, Loss: 0.0199
Training Zone 2 in City1...
Epoch 1/2, Loss: 0.0199
Epoch 2/2, Loss: 0.0191
Training Zone 3 in City1...
Epoch 1/2, Loss: 0.0210
Epoch 2/2, Loss: 0.0203
Training Zone 1 in City2...
Epoch 1/2, Loss: 0.0238
Epoch 2/2, Loss: 0.0229
Training Zone 2 in City2...
Epoch 1/2, Loss: 0.0199
Epoch 2/2, Loss: 0.0194
Training Zone 3 in City2...
Epoch 1/2, Loss: 0.0206
Epoch 2/2, Loss: 0.0197
Training Zone 1 in City3...
Epoch 1/2, Loss: 0.0306
Epoch 2/2, Loss: 0.0301


In [2]:
import os
import pandas as pd
import numpy as np
import mindspore as ms
from mindspore import nn, Tensor
from mindspore.dataset import GeneratorDataset

# -----------------------------
# Setup
# -----------------------------
ms.set_seed(42)
np.random.seed(42)
ms.set_context(mode=ms.PYNATIVE_MODE)

# -----------------------------
# LSTM Model
# -----------------------------
class LSTMModel(nn.Cell):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Dense(hidden_size, output_size)

        self._input_size = input_size
        self._hidden_size = hidden_size
        self._num_layers = num_layers
        self._output_size = output_size

    def construct(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

# -----------------------------
# Dataset
# -----------------------------
class PharmacyDataset:
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return (
            np.array(self.sequences[idx], dtype=np.float32),
            np.array(self.targets[idx], dtype=np.float32)
        )

# -----------------------------
# Load Data
# -----------------------------
def load_data(file_path, global_columns=None):
    data = pd.read_csv(file_path)

    data['addeddate'] = pd.to_datetime(
        data['addeddate'].astype(str).str.split().str[0],
        errors='coerce'
    )
    data.dropna(subset=['addeddate'], inplace=True)

    data['month'] = data['addeddate'].dt.to_period('M')
    data['medication_combination'] = data.groupby('Invoice')['name'].transform(
        lambda x: '+'.join(sorted(set(x)))
    )

    grouped = data.groupby(['month', 'medication_combination']).size().unstack(fill_value=0)

    if global_columns is not None:
        grouped = grouped.reindex(columns=global_columns, fill_value=0)

    return grouped

# -----------------------------
# Create sequences
# -----------------------------
def create_sequences(data, seq_length=3):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i + seq_length].values)
        y.append(data.iloc[i + seq_length].values)
    return X, y

# -----------------------------
# Train model
# -----------------------------
def train_model(model, dataset, loss_fn, optimizer, epochs=2):
    model.set_train(True)

    def forward_fn(x, y):
        return loss_fn(model(x), y)

    grad_fn = ms.value_and_grad(forward_fn, None, optimizer.parameters)

    for e in range(epochs):
        total, steps = 0.0, 0
        for xb, yb in dataset.create_tuple_iterator():
            xb, yb = Tensor(xb, ms.float32), Tensor(yb, ms.float32)
            loss, grads = grad_fn(xb, yb)
            optimizer(grads)
            total += float(loss.asnumpy())
            steps += 1
        print(f"Epoch {e+1}/{epochs}, Loss: {total/max(steps,1):.4f}")

# -----------------------------
# Aggregate models (average)
# -----------------------------
def aggregate_models(models):
    base = models[0]
    agg = LSTMModel(
        base._input_size, base._hidden_size,
        base._num_layers, base._output_size
    )

    param_sets = [{p.name: p.data.asnumpy() for p in m.get_parameters()} for m in models]

    for p in agg.get_parameters():
        p.set_data(
            Tensor(np.mean([ps[p.name] for ps in param_sets], axis=0), p.data.dtype)
        )

    return agg

# -----------------------------
# Predict
# -----------------------------
def predict_trends(model, data, seq_length=3, steps=3):
    model.set_train(False)
    x = Tensor(data.iloc[-seq_length:].values.astype(np.float32)).expand_dims(0)

    preds = []
    for _ in range(steps):
        out = model(x).asnumpy()
        preds.append(out)
        x = ms.ops.concat((x[:, 1:, :], Tensor(out).expand_dims(1)), axis=1)

    return np.array(preds)

# -----------------------------
# MAIN
# -----------------------------
if __name__ == "__main__":

    input_root = "./outputDateUni/"
    result_root = "./Real_LSTM_Result/"
    os.makedirs(result_root, exist_ok=True)

    cities = ["City1", "City2", "City3"]
    zones_per_city = 3
    pharmacies_per_zone = 4
    seq_length = 3

    # -----------------------------
    # Global columns
    # -----------------------------
    all_cols = set()
    for city in cities:
        for z in range(1, zones_per_city + 1):
            for p in range(1, pharmacies_per_zone + 1):
                path = os.path.join(
                    input_root, city, f"Zone{z}",
                    f"Ph{p:02d}_Z{z:02d}_C{cities.index(city)+1:02d}.csv"
                )
                if os.path.exists(path):
                    d = pd.read_csv(path)
                    d['medication_combination'] = d.groupby('Invoice')['name'].transform(
                        lambda x: '+'.join(sorted(set(x)))
                    )
                    all_cols.update(d['medication_combination'].unique())

    global_columns = sorted(all_cols)

    # -----------------------------
    # Train zones & aggregate per city
    # -----------------------------
    for city in cities:
        print(f"\n=== Processing {city} ===")
        zone_models = []
        zone_data = []

        for z in range(1, zones_per_city + 1):
            dfs = []
            for p in range(1, pharmacies_per_zone + 1):
                path = os.path.join(
                    input_root, city, f"Zone{z}",
                    f"Ph{p:02d}_Z{z:02d}_C{cities.index(city)+1:02d}.csv"
                )
                if os.path.exists(path):
                    dfs.append(load_data(path, global_columns))

            if not dfs:
                continue

            merged = sum(dfs) / len(dfs)
            X, y = create_sequences(merged, seq_length)
            if not X:
                continue

            dataset = GeneratorDataset(
                PharmacyDataset(X, y),
                ["x", "y"], shuffle=True
            ).batch(32)

            model = LSTMModel(
                merged.shape[1], 64, 2, merged.shape[1]
            )

            opt = nn.Adam(model.trainable_params(), learning_rate=0.001)
            loss = nn.MSELoss()

            print(f"Training {city} Zone {z}")
            train_model(model, dataset, loss, opt)

            zone_models.append(model)
            zone_data.append(merged)

        # -----------------------------
        # City aggregation
        # -----------------------------
        if zone_models:
            city_model = aggregate_models(zone_models)
            city_data = sum(zone_data) / len(zone_data)

            preds = predict_trends(city_model, city_data)
            df = pd.DataFrame(preds.squeeze(), columns=city_data.columns)

            out_path = os.path.join(result_root, f"{city}_predictions.csv")
            df.to_csv(out_path, index=False)

            print(f"✔ City predictions saved → {out_path}")

    print("\n✅ Finished: City-level HFL predictions generated.")



=== Processing City1 ===
Training City1 Zone 1
Epoch 1/2, Loss: 0.0213
Epoch 2/2, Loss: 0.0199
Training City1 Zone 2
Epoch 1/2, Loss: 0.0199
Epoch 2/2, Loss: 0.0191
Training City1 Zone 3
Epoch 1/2, Loss: 0.0210
Epoch 2/2, Loss: 0.0203
✔ City predictions saved → ./Real_LSTM_Result/City1_predictions.csv

=== Processing City2 ===
Training City2 Zone 1
Epoch 1/2, Loss: 0.0236
Epoch 2/2, Loss: 0.0229
Training City2 Zone 2
Epoch 1/2, Loss: 0.0199
Epoch 2/2, Loss: 0.0192
Training City2 Zone 3
Epoch 1/2, Loss: 0.0205
Epoch 2/2, Loss: 0.0198
✔ City predictions saved → ./Real_LSTM_Result/City2_predictions.csv

=== Processing City3 ===
Training City3 Zone 1
Epoch 1/2, Loss: 0.0307
Epoch 2/2, Loss: 0.0300


MemoryError: Unable to allocate 833. MiB for an array with shape (256, 426247) and data type float64

In [3]:
import os
import pandas as pd
import numpy as np
import mindspore as ms
from mindspore import nn, Tensor
from mindspore.dataset import GeneratorDataset

# -----------------------------
# Setup
# -----------------------------
ms.set_seed(42)
np.random.seed(42)
ms.set_context(mode=ms.PYNATIVE_MODE)

# >>> QUICK FIX KNOB <<<
TOP_K = 500   # try 5000 first; if still heavy, use 2000-3000; if you have more RAM, use 10000.

# -----------------------------
# LSTM Model
# -----------------------------
class LSTMModel(nn.Cell):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Dense(hidden_size, output_size)

        self._input_size = input_size
        self._hidden_size = hidden_size
        self._num_layers = num_layers
        self._output_size = output_size

    def construct(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

# -----------------------------
# Dataset
# -----------------------------
class PharmacyDataset:
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return (
            np.array(self.sequences[idx], dtype=np.float32),
            np.array(self.targets[idx], dtype=np.float32)
        )

# -----------------------------
# Load Data
# -----------------------------
def load_data(file_path, global_columns=None):
    data = pd.read_csv(file_path)

    data['addeddate'] = pd.to_datetime(
        data['addeddate'].astype(str).str.split().str[0],
        errors='coerce'
    )
    data.dropna(subset=['addeddate'], inplace=True)

    data['month'] = data['addeddate'].dt.to_period('M')
    data['medication_combination'] = data.groupby('Invoice')['name'].transform(
        lambda x: '+'.join(sorted(set(x)))
    )

    grouped = data.groupby(['month', 'medication_combination']).size().unstack(fill_value=0)

    if global_columns is not None:
        grouped = grouped.reindex(columns=global_columns, fill_value=0)

    return grouped

# -----------------------------
# Create sequences
# -----------------------------
def create_sequences(data, seq_length=3):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i + seq_length].values)
        y.append(data.iloc[i + seq_length].values)
    return X, y

# -----------------------------
# Train model
# -----------------------------
def train_model(model, dataset, loss_fn, optimizer, epochs=2):
    model.set_train(True)

    def forward_fn(x, y):
        return loss_fn(model(x), y)

    grad_fn = ms.value_and_grad(forward_fn, None, optimizer.parameters)

    for e in range(epochs):
        total, steps = 0.0, 0
        for xb, yb in dataset.create_tuple_iterator():
            xb, yb = Tensor(xb, ms.float32), Tensor(yb, ms.float32)
            loss, grads = grad_fn(xb, yb)
            optimizer(grads)
            total += float(loss.asnumpy())
            steps += 1
        print(f"Epoch {e+1}/{epochs}, Loss: {total/max(steps,1):.4f}")

# -----------------------------
# Aggregate models (average)
# -----------------------------
def aggregate_models(models):
    base = models[0]
    agg = LSTMModel(
        base._input_size, base._hidden_size,
        base._num_layers, base._output_size
    )

    param_sets = [{p.name: p.data.asnumpy() for p in m.get_parameters()} for m in models]

    for p in agg.get_parameters():
        # Average parameter tensors across models
        stacked = np.mean([ps[p.name] for ps in param_sets], axis=0)
        p.set_data(Tensor(stacked, p.data.dtype))

    return agg

# -----------------------------
# Predict
# -----------------------------
def predict_trends(model, data, seq_length=3, steps=3):
    model.set_train(False)
    x = Tensor(data.iloc[-seq_length:].values.astype(np.float32)).expand_dims(0)

    preds = []
    for _ in range(steps):
        out = model(x).asnumpy()
        preds.append(out)
        x = ms.ops.concat((x[:, 1:, :], Tensor(out.astype(np.float32)).expand_dims(1)), axis=1)

    return np.array(preds)

# -----------------------------
# Build Top-K global columns by frequency
# -----------------------------
def build_topk_global_columns(input_root, cities, zones_per_city, pharmacies_per_zone, top_k):
    # Count frequency of medication combinations across all files
    counts = {}

    for city in cities:
        for z in range(1, zones_per_city + 1):
            for p in range(1, pharmacies_per_zone + 1):
                path = os.path.join(
                    input_root, city, f"Zone{z}",
                    f"Ph{p:02d}_Z{z:02d}_C{cities.index(city)+1:02d}.csv"
                )
                if not os.path.exists(path):
                    continue

                d = pd.read_csv(path)
                if "Invoice" not in d.columns or "name" not in d.columns:
                    continue

                combos = d.groupby("Invoice")["name"].apply(lambda x: '+'.join(sorted(set(x))))
                vc = combos.value_counts()

                for k, v in vc.items():
                    counts[k] = counts.get(k, 0) + int(v)

    if not counts:
        return []

    # Take Top-K most frequent combinations
    top = sorted(counts.items(), key=lambda x: x[1], reverse=True)[:top_k]
    global_cols = [k for k, _ in top]
    return global_cols

# -----------------------------
# MAIN
# -----------------------------
if __name__ == "__main__":

    input_root = "./outputDateUni/"
    result_root = "./Real_LSTM_Result/"
    os.makedirs(result_root, exist_ok=True)

    cities = ["City1", "City2", "City3"]
    zones_per_city = 3
    pharmacies_per_zone = 4
    seq_length = 3

    print(f"Building Top-{TOP_K} global columns (this prevents feature explosion)...")
    global_columns = build_topk_global_columns(
        input_root, cities, zones_per_city, pharmacies_per_zone, TOP_K
    )

    if not global_columns:
        raise RuntimeError("No global columns were built. Check your dataset paths/columns (Invoice/name).")

    print(f"Using global feature size = {len(global_columns)} columns")

    # -----------------------------
    # Train zones & aggregate per city
    # -----------------------------
    for city in cities:
        print(f"\n=== Processing {city} ===")
        zone_models = []
        zone_data = []

        for z in range(1, zones_per_city + 1):
            dfs = []
            for p in range(1, pharmacies_per_zone + 1):
                path = os.path.join(
                    input_root, city, f"Zone{z}",
                    f"Ph{p:02d}_Z{z:02d}_C{cities.index(city)+1:02d}.csv"
                )
                if os.path.exists(path):
                    dfs.append(load_data(path, global_columns))

            if not dfs:
                continue

            merged = sum(dfs) / len(dfs)

            # If still too wide (unlikely now), hard-cap columns (extra safety)
            if merged.shape[1] > TOP_K:
                merged = merged.iloc[:, :TOP_K]

            X, y = create_sequences(merged, seq_length)
            if not X:
                continue

            dataset = GeneratorDataset(
                PharmacyDataset(X, y),
                ["x", "y"], shuffle=True
            ).batch(32)

            model = LSTMModel(
                merged.shape[1], 64, 2, merged.shape[1]
            )

            opt = nn.Adam(model.trainable_params(), learning_rate=0.001)
            loss = nn.MSELoss()

            print(f"Training {city} Zone {z} (features={merged.shape[1]})")
            train_model(model, dataset, loss, opt, epochs=2)

            zone_models.append(model)
            zone_data.append(merged)

        # -----------------------------
        # City aggregation + City prediction CSV
        # -----------------------------
        if zone_models:
            city_model = aggregate_models(zone_models)
            city_data = sum(zone_data) / len(zone_data)

            preds = predict_trends(city_model, city_data)
            df = pd.DataFrame(preds.squeeze(), columns=city_data.columns)

            out_path = os.path.join(result_root, f"{city}_predictions.csv")
            df.to_csv(out_path, index=False)
            print(f"✔ City predictions saved → {out_path}")
        else:
            print(f"⚠ No trained zones found for {city}. No CSV generated.")

    print("\n✅ Finished: City-level predictions generated (national skipped).")


Building Top-500 global columns (this prevents feature explosion)...
Using global feature size = 500 columns

=== Processing City1 ===
Training City1 Zone 1 (features=500)
Epoch 1/2, Loss: 2.3103
Epoch 2/2, Loss: 2.2883
Training City1 Zone 2 (features=500)
Epoch 1/2, Loss: 2.2913
Epoch 2/2, Loss: 2.2625
Training City1 Zone 3 (features=500)
Epoch 1/2, Loss: 2.2660
Epoch 2/2, Loss: 2.2396
✔ City predictions saved → ./Real_LSTM_Result/City1_predictions.csv

=== Processing City2 ===
Training City2 Zone 1 (features=500)
Epoch 1/2, Loss: 2.2931
Epoch 2/2, Loss: 2.2672
Training City2 Zone 2 (features=500)
Epoch 1/2, Loss: 2.2232
Epoch 2/2, Loss: 2.1972
Training City2 Zone 3 (features=500)
Epoch 1/2, Loss: 2.2627
Epoch 2/2, Loss: 2.2427
✔ City predictions saved → ./Real_LSTM_Result/City2_predictions.csv

=== Processing City3 ===
Training City3 Zone 1 (features=500)
Epoch 1/2, Loss: 2.2515
Epoch 2/2, Loss: 2.2252
Training City3 Zone 2 (features=500)
Epoch 1/2, Loss: 2.2695
Epoch 2/2, Loss: 2.24